# BatchWrite Example

### Housekeeping - Setup & wipe out any prior records on the Aerospike Server

We have a namespace **_test_** pre-defined on the server. Lets truncate it using _asadm_.

This is needed while doing code development. If you want to clear the iJava Kernel of all Java objects and run all cells from scratch, Kernel->Restart & Run All, this will ensure any records written on the underlying Aerospike cluster are purged.

First, we need required imports for using %sh in interactive Java Kernel. (This is specific to the iJava Kernel implementation by **Spencer Park** that we are using.)

In [1]:
import io.github.spencerpark.ijava.IJava;
import io.github.spencerpark.jupyter.kernel.magic.common.Shell;
IJava.getKernelInstance().getMagics().registerMagics(Shell.class);

### Running _asadm_ in iJava
We can run _asadm_ commands inline. Below, we will use the truncation command, which normally requires an interactive confirmation, which we will skip by using the _--no-warn_ flag. No output will be displayed.

In [2]:
%sh asadm --enable -e "manage truncate ns test --no-warn" -h "127.0.0.1"

#### Add Java Client POM Dependency  
Jupyter Notebook way!

In [3]:
%%loadFromPOM
<dependencies>
  <dependency>
    <groupId>com.aerospike</groupId>
    <artifactId>aerospike-client-jdk8</artifactId>
    <version>8.1.1</version>
  </dependency>
</dependencies>

####  Java Client Imports and Test code

In [4]:
//Required Imports
import com.aerospike.client.AerospikeClient;
import com.aerospike.client.policy.WritePolicy;
import com.aerospike.client.Bin;
import com.aerospike.client.Key;
import com.aerospike.client.Record;
import com.aerospike.client.Value;

//import com.aerospike.client.cdt.ListPolicy;
//import com.aerospike.client.cdt.ListOperation;
//import com.aerospike.client.cdt.ListOrder;
//import com.aerospike.client.cdt.ListWriteFlags;
//import com.aerospike.client.cdt.ListReturnType;

//import com.aerospike.client.cdt.MapPolicy;
//import com.aerospike.client.cdt.MapOrder;
//import com.aerospike.client.cdt.MapWriteFlags;
//import com.aerospike.client.cdt.MapReturnType;

//import com.aerospike.client.exp.MapExp;
//import com.aerospike.client.cdt.CTX;
//import java.util.HashMap;
//import java.util.Map;

import com.aerospike.client.Operation;
//import com.aerospike.client.query.Statement;
//import com.aerospike.client.exp.Exp;
//import com.aerospike.client.cdt.MapOperation;

import com.aerospike.client.policy.BatchPolicy;
import com.aerospike.client.policy.BatchWritePolicy;
import com.aerospike.client.BatchResults;
import com.aerospike.client.BatchRecord;
import com.aerospike.client.ResultCode;

//import java.util.Random;



In [5]:
AerospikeClient client = new AerospikeClient("127.0.0.1", 3000);


In [6]:
Key key1 = new Key("test", "testSet", 1); 
Key key2 = new Key("test", "testSet", 2); 
Key key3 = new Key("test", "testSet", 3);


In [7]:
BatchPolicy batchPolicy = new BatchPolicy();
//batchPolicy.sendKey = true;
BatchWritePolicy batchWritePolicy = new BatchWritePolicy();
//batchWritePolicy.sendKey = true;
        
//client.operate(batchPolicy, batchWritePolicy, new Key[] {key1, key2, key3}, Operation.put(new Bin("now", new Date().getTime())));
BatchResults bresults = client.operate(batchPolicy, batchWritePolicy, 
                                       new Key[] {key1, key2, key3}, 
                                       new Operation [] {
                                         Operation.add(new Bin("bin1", 1)),
                                         Operation.put(new Bin("bin2", "CA")),
                                         Operation.put(new Bin("bin3", 301))}
                                       );

System.out.println("Batch Write Status: " + bresults.status);

// process the BatchResults returned from the batch operation
for (int i = 0; i < bresults.records.length; i++) {
    BatchRecord br = bresults.records[i];
    Record rec = br.record;
    if (br.resultCode == ResultCode.OK) {          // check individual key status 
        long bin1Val = rec.getLong("bin1");        // bin1 has one operation, op result directly accessible
        // note the result order within each list matches ops order for the bin
        System.out.format("Result[%d]: key: %s/%s/%s, bin1: %d\n",i, br.key.namespace, br.key.setName, br.key.userKey, bin1Val);
    }
    else {   // error in individual key's operations
        System.out.format("Result[%d]: key: %s, error: %s\n", 
                            i, br.key, ResultCode.getResultString(br.resultCode));
    }
}


Batch Write Status: true
Result[0]: key: test/testSet/1, bin1: 0
Result[1]: key: test/testSet/2, bin1: 0
Result[2]: key: test/testSet/3, bin1: 0


In [8]:
System.out.println(client.get(null, key1));
System.out.println(client.get(null, key2));
System.out.println(client.get(null, key3));

(gen:1),(exp:450895691),(bins:(bin1:1),(bin2:CA),(bin3:301))
(gen:1),(exp:450895691),(bins:(bin1:1),(bin2:CA),(bin3:301))
(gen:1),(exp:450895691),(bins:(bin1:1),(bin2:CA),(bin3:301))


In [13]:
import com.aerospike.client.BatchRecord;
import com.aerospike.client.BatchResults;
import com.aerospike.client.BatchWrite;

Key key1 = new Key("test", "testSet", 1); 
Key key2 = new Key("test", "testSet", 2); 
Key key3 = new Key("test", "testSet", 3);

BatchPolicy batchPolicy = new BatchPolicy();
BatchWritePolicy batchWritePolicy = new BatchWritePolicy();

//We will specify what to do for each record in this list
List<BatchRecord> batchRecords = new ArrayList<BatchRecord>();

// Operations to do on key1
Operation [] key1ops = Operation.array(
                        Operation.put(new Bin("bin1", 111)),
                        Operation.put(new Bin("bin2", "MA")),
                        Operation.put(new Bin("bin3", 311))
                       );
                       
                       
batchRecords.add(new BatchWrite(key1, key1ops)); 

// Operations to do on key2  (add in bin2, type string, error expected)
Operation [] key2ops = Operation.array(
                        Operation.put(new Bin("bin1", 122)),
                        Operation.add(new Bin("bin2", 1)),
                        Operation.put(new Bin("bin3", 322))
                       );
                       
                       
batchRecords.add(new BatchWrite(key2, key2ops)); 

// Operations to do on key3
Operation [] key3ops = Operation.array(
                        Operation.put(new Bin("bin1", 133)),
                        Operation.put(new Bin("bin2", "TX")),
                        Operation.put(new Bin("bin3", 333))
                       );
                       
                       
batchRecords.add(new BatchWrite(key3, key3ops)); 

boolean bDone = client.operate(batchPolicy, batchRecords);

System.out.println("Batch Write Status: " + bDone);

// Individual results of Write operations with BatchWrite.
for(int i = 0; i<3; i++){
  BatchRecord batchRec = batchRecords.get(i);
  Record rec = batchRec.record;
  Key key = batchRec.key;
  if (batchRec.resultCode == ResultCode.OK) {
    System.out.format("Result[%d]: %s\n",i, client.get(null, new Key("test", "testSet", (i+1))));
  }
  else {
    System.out.format("Result[%d]: error: %s\n", i, ResultCode.getResultString(batchRec.resultCode));
  }
}


Batch Write Status: false
Result[0]: (gen:6),(exp:450896090),(bins:(bin1:111),(bin2:MA),(bin3:311))
Result[1]: error: Bin type error
Result[2]: (gen:6),(exp:450896090),(bins:(bin1:133),(bin2:TX),(bin3:333))


In [14]:
System.out.println(client.get(null, key1));
System.out.println(client.get(null, key2));
System.out.println(client.get(null, key3));

(gen:6),(exp:450896090),(bins:(bin1:111),(bin2:MA),(bin3:311))
(gen:1),(exp:450895691),(bins:(bin1:1),(bin2:CA),(bin3:301))
(gen:6),(exp:450896090),(bins:(bin1:133),(bin2:TX),(bin3:333))


# Cleanup

In [15]:
%sh asadm --enable -e "manage truncate ns test --no-warn" -h "127.0.0.1"